<a href="https://colab.research.google.com/github/davidlealo/asistenteabp/blob/main/agenteabp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalar bibliotecas necesarias

In [1]:
! pip install mistralai
!pip install transformers ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.0/260.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.5 MB/s eta 0:00:00


# Solicitar Clave Mistral

In [2]:
from IPython.display import display, HTML, Javascript
from transformers import AutoModelForCausalLM, AutoTokenizer

# Interfaz para ingresar la API key
def request_api_key():
    input_html = """
    <div style="font-family: Arial; margin: 10px; padding: 10px; border: 1px solid #ddd; border-radius: 5px;">
        <h3>Proveer API Key</h3>
        <p>Por favor, ingrese su API key de Mistral en el siguiente campo:</p>
        <input type="password" id="apiKeyInput" placeholder="Ingrese su API key aquí" style="width: 100%; padding: 8px; margin-bottom: 10px;">
        <button onclick="storeApiKey()" style="padding: 10px 15px; background-color: #007BFF; color: white; border: none; border-radius: 3px; cursor: pointer;">Guardar API Key</button>
    </div>
    <script>
        function storeApiKey() {
            const apiKey = document.getElementById('apiKeyInput').value;
            if (apiKey) {
                google.colab.kernel.invokeFunction('notebook.store_api_key', [apiKey], {});
                alert('API key almacenada correctamente.');
            } else {
                alert('Por favor, ingrese una API key válida.');
            }
        }
    </script>
    """
    display(HTML(input_html))

# Callback para almacenar la API key en el entorno Python
def store_api_key(api_key):
    global mistral_api_key
    mistral_api_key = api_key
    print("API key almacenada correctamente.")

# Registrar la función en Google Colab
from google.colab import output
output.register_callback('notebook.store_api_key', store_api_key)

# Solicitar la API key al usuario
request_api_key()


API key almacenada correctamente.


# Carga nodelo de Mistral

In [3]:
import requests

# Define la función para interactuar con la API de Mistral
def run_mistral(user_message, model="mistral-large-latest", api_key=None):
    if not api_key:
        raise ValueError("Por favor, proporciona una API key válida para Mistral.")

    # URL base de la API (esto puede variar dependiendo de la documentación de Mistral)
    api_url = f"https://api.mistral.ai/v1/chat/completions"

    # Configura el payload para la solicitud
    payload = {
        "model": model,
        "messages": [
            {"role": "user", "content": user_message}
        ]
    }

    # Cabeceras de autenticación
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    # Envía la solicitud POST
    response = requests.post(api_url, json=payload, headers=headers)

    # Manejo de errores
    if response.status_code != 200:
        raise Exception(f"Error en la API de Mistral: {response.status_code} {response.text}")

    # Devuelve la respuesta generada
    response_data = response.json()
    return response_data["choices"][0]["message"]["content"]



# Prompt master del agente

In [4]:
!wget https://raw.githubusercontent.com/davidlealo/asistenteabp/refs/heads/main/prompt.xml

--2024-12-12 18:50:21--  https://raw.githubusercontent.com/davidlealo/asistenteabp/refs/heads/main/prompt.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5176 (5.1K) [text/plain]
Saving to: ‘prompt.xml’

prompt.xml          100%[===================>]   5.05K  --.-KB/s    in 0s      

2024-12-12 18:50:22 (40.7 MB/s) - ‘prompt.xml’ saved [5176/5176]



# Interactuar con el agente

In [7]:
from IPython.display import display, HTML, Markdown
from google.colab import output
import requests
import xml.etree.ElementTree as ET

class AgenteABPConfiguracion:
    def __init__(self, xml_path):
        self.config = self.cargar_configuracion(xml_path)

    def cargar_configuracion(self, xml_path):
        tree = ET.parse(xml_path)
        root = tree.getroot()
        config = {}
        config['description'] = root.find('description').text
        config['warning'] = root.find('warning').text
        config['context'] = root.find('context').find('languageSpecification').text
        return config

    def crear_contexto(self, datos_proyecto):
        contexto = f"{self.config['description']}\n\n"
        contexto += f"{self.config['warning']}\n\n"
        contexto += "Detalles del Proyecto:\n"
        for clave, valor in datos_proyecto.items():
            contexto += f"- {clave}: {valor}\n"
        return contexto

# Configuración del agente
agente = AgenteABPConfiguracion('prompt.xml')

# Verifica si la API key ya está almacenada
if 'mistral_api_key' not in globals():
    print("⚠️ Por favor, asegúrate de haber configurado tu API key antes de continuar.")

# Solicitar datos del proyecto al usuario
def request_project_details():
    html = """
    <div style="font-family: Arial, sans-serif; padding: 10px; max-width: 600px; margin: auto;">
        <h3 style="color: #2c3e50;">Detalles del Proyecto</h3>
        <label style="font-weight: bold;">Objetivos de Aprendizaje:</label><br>
        <textarea id="learning_objectives" style="width: 100%; height: 80px; border: 1px solid #ccc; border-radius: 5px; padding: 5px;"></textarea><br><br>
        <label style="font-weight: bold;">Asignaturas involucradas:</label><br>
        <textarea id="subjects" style="width: 100%; height: 80px; border: 1px solid #ccc; border-radius: 5px; padding: 5px;"></textarea><br><br>
        <label style="font-weight: bold;">Otros detalles relevantes:</label><br>
        <textarea id="details" style="width: 100%; height: 80px; border: 1px solid #ccc; border-radius: 5px; padding: 5px;"></textarea><br><br>
        <button style="background-color: #3498db; color: white; padding: 10px 15px; border: none; border-radius: 5px; cursor: pointer;"
                onclick="submitDetails()">Generar Proyecto</button>
    </div>
    <script>
        function submitDetails() {
            const learningObjectives = document.getElementById("learning_objectives").value;
            const subjects = document.getElementById("subjects").value;
            const details = document.getElementById("details").value;
            google.colab.kernel.invokeFunction("notebook.generate_project", [learningObjectives, subjects, details], {});
        }
    </script>
    """
    display(HTML(html))

# Callback para generar el proyecto
def generate_project(learning_objectives, subjects, details):
    if 'mistral_api_key' not in globals() or not mistral_api_key:
        print("⚠️ Error: No se encontró una API key válida. Por favor, configura tu API key.")
        return

    # Usar el agente para estructurar el prompt
    agente_contexto = agente.crear_contexto({
        "Objetivos de Aprendizaje": learning_objectives,
        "Asignaturas": subjects,
        "Detalles Adicionales": details
    })

    # Crear el prompt para Mistral usando el agente
    prompt = f"""
    {agente_contexto}
    Sigue el estándar del Gold Standard de PBLWorks y proporciona:
    - Título del Proyecto
    - Pregunta guía
    - Fase 1: Construir Conocimiento, Comprensión y Habilidades
    - Fase 2: Desarrollar y Evaluar Productos y Respuestas
    - Fase 3: Presentar Productos y Respuestas

    Nota: Asegúrate de incluir elementos de autenticidad, reflexión y retroalimentación.
    """

    # Llamar a la API de Mistral
    headers = {
        "Authorization": f"Bearer {mistral_api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "mistral-large-latest",
        "messages": [{"role": "user", "content": prompt}]
    }
    response = requests.post("https://api.mistral.ai/v1/chat/completions", json=data, headers=headers)

    # Procesar respuesta
    if response.status_code == 200:
        project = response.json().get("choices", [{}])[0].get("message", {}).get("content", "No se generó contenido.")
        # Mostrar el proyecto en formato Markdown
        display(Markdown(f"# Proyecto Generado\n\n{project}"))
    else:
        print(f"❌ Error al generar el proyecto: {response.status_code} - {response.text}")

# Registrar la función en Colab
output.register_callback("notebook.generate_project", generate_project)

# Solicitar detalles del proyecto al usuario
request_project_details()


# Proyecto Generado

### Agente de Guía para el Diseño de Proyectos ABP

#### Título del Proyecto: "Dilemas Éticos y Soluciones Matemáticas: Una Mirada Filosófica y Tecnológica"

#### Pregunta Guía:
"¿Cómo pueden los principios filosóficos y las herramientas matemáticas ayudar a abordar dilemas éticos contemporáneos en nuestra sociedad y tecnología?"

---

### Fase 1: Construir Conocimiento, Comprensión y Habilidades

#### Objetivos de Aprendizaje:
1. Desarrollar un pensamiento crítico y reflexivo frente a problemas filosóficos contemporáneos.
2. Analizar textos filosóficos y evaluar argumentaciones.
3. Resolver problemas matemáticos avanzados y modelar situaciones del mundo real.
4. Aplicar métodos estadísticos para recolectar, analizar e interpretar información.
5. Utilizar herramientas digitales para representar y resolver problemas matemáticos.
6. Desarrollar habilidades de comunicación oral y escrita en inglés.

#### Actividades y Recursos:
1. **Lecturas y Discusiones Filosóficas**:
   - Introducción a textos filosóficos clásicos y contemporáneos.
   - Discusiones en grupo sobre ética, sociedad y tecnología.
   - Evaluación de argumentaciones y conexión con situaciones reales.

2. **Talleres Matemáticos**:
   - Resolución de problemas relacionados con funciones avanzadas (polinomios, logaritmos).
   - Modelado de situaciones del mundo real utilizando conceptos de optimización y análisis de datos.
   - Uso de herramientas digitales como GeoGebra, Python o Excel para representar y resolver problemas matemáticos.

3. **Métodos Estadísticos**:
   - Introducción a métodos estadísticos básicos y avanzados.
   - Recolección, análisis e interpretación de datos.
   - Aplicación de estadísticas en investigaciones interdisciplinarias.

4. **Desarrollo de Habilidades en Inglés**:
   - Lectura y comprensión de textos informativos y argumentativos en inglés.
   - Redacción de informes o ensayos en inglés.
   - Presentaciones orales en inglés sobre problemas globales.

---

### Fase 2: Desarrollar y Evaluar Productos y Respuestas

#### Actividades y Recursos:
1. **Proyecto Interdisciplinario**:
   - Los estudiantes trabajarán en grupos para desarrollar un proyecto que aborde un dilema ético contemporáneo.
   - Cada grupo debe identificar un problema específico y aplicar principios filosóficos y herramientas matemáticas para proponer soluciones.

2. **Investigación y Análisis**:
   - Recolección de datos relevantes utilizando métodos estadísticos.
   - Análisis de datos y modelado matemático.
   - Evaluación de soluciones propuestas.

3. **Desarrollo de Productos**:
   - Creación de informes o ensayos en inglés que documenten el proceso y las soluciones propuestas.
   - Desarrollo de presentaciones orales en inglés para comunicar los hallazgos.

4. **Retroalimentación y Reflexión**:
   - Sesiones de retroalimentación en grupo para evaluar el progreso y ajustar las estrategias.
   - Reflexión individual y grupal sobre el proceso y los aprendizajes adquiridos.

---

### Fase 3: Presentar Productos y Respuestas

#### Actividades y Recursos:
1. **Presentación de Proyectos**:
   - Presentaciones orales de los proyectos en inglés.
   - Uso de herramientas digitales para apoyar las presentaciones (PowerPoint, Prezi, etc.).

2. **Evaluación y Retroalimentación**:
   - Evaluación de los proyectos por parte de los docentes y pares.
   - Retroalimentación constructiva y reflexión sobre el proceso y los resultados.

3. **Publicación y Difusión**:
   - Publicación de los informes y ensayos en una plataforma digital o en una revista escolar.
   - Difusión de los proyectos en eventos escolares o comunitarios.

---

### Elementos de Autenticidad, Reflexión y Retroalimentación

1. **Autenticidad**:
   - Los proyectos deben estar alineados con problemas reales y contemporáneos.
   - Uso de herramientas y métodos que se utilizan en el mundo profesional.

2. **Reflexión**:
   - Diarios de reflexión para que los estudiantes documenten su proceso de aprendizaje.
   - Sesiones de reflexión grupal para evaluar el progreso y los aprendizajes.

3. **Retroalimentación**:
   - Retroalimentación continua por parte de los docentes y pares.
   - Uso de rúbricas y criterios claros para la evaluación de los proyectos.

---

Este enfoque asegura que los proyectos estén alineados con el Gold Standard de PBLWorks y sigan el Project Path, fomentando habilidades del siglo XXI y la innovación pedagógica.

# Segunda versión agente

In [8]:
from IPython.display import display, HTML, Markdown
from google.colab import output
import requests
import xml.etree.ElementTree as ET

class AgenteABPConfiguracion:
    def __init__(self, xml_path):
        self.config = self.cargar_configuracion(xml_path)

    def cargar_configuracion(self, xml_path):
        tree = ET.parse(xml_path)
        root = tree.getroot()
        config = {}
        config['description'] = root.find('description').text
        config['warning'] = root.find('warning').text
        config['context'] = root.find('context').find('languageSpecification').text
        return config

    def crear_contexto(self, datos_proyecto):
        contexto = f"{self.config['description']}\n\n"
        contexto += f"{self.config['warning']}\n\n"
        contexto += "Detalles del Proyecto:\n"
        for clave, valor in datos_proyecto.items():
            contexto += f"- {clave}: {valor}\n"
        return contexto

# Configuración del agente
agente = AgenteABPConfiguracion('prompt.xml')

# Verifica si la API key ya está almacenada
if 'mistral_api_key' not in globals():
    print("⚠️ Por favor, asegúrate de haber configurado tu API key antes de continuar.")

# Solicitar datos del proyecto al usuario
def request_project_details():
    html = """
    <div style="font-family: Arial, sans-serif; padding: 10px; max-width: 600px; margin: auto;">
        <h3 style="color: #2c3e50;">Detalles del Proyecto</h3>
        <label style="font-weight: bold;">Objetivos de Aprendizaje:</label><br>
        <textarea id="learning_objectives" style="width: 100%; height: 80px; border: 1px solid #ccc; border-radius: 5px; padding: 5px;"></textarea><br><br>
        <label style="font-weight: bold;">Asignaturas involucradas:</label><br>
        <textarea id="subjects" style="width: 100%; height: 80px; border: 1px solid #ccc; border-radius: 5px; padding: 5px;"></textarea><br><br>
        <label style="font-weight: bold;">Otros detalles relevantes:</label><br>
        <textarea id="details" style="width: 100%; height: 80px; border: 1px solid #ccc; border-radius: 5px; padding: 5px;"></textarea><br><br>
        <button style="background-color: #3498db; color: white; padding: 10px 15px; border: none; border-radius: 5px; cursor: pointer;"
                onclick="submitDetails()">Generar Proyecto</button>
        <div id="loading" style="display: none; margin-top: 20px; font-weight: bold; color: #3498db;">Procesando... Por favor, espera.</div>
    </div>
    <script>
        function submitDetails() {
            const learningObjectives = document.getElementById("learning_objectives").value;
            const subjects = document.getElementById("subjects").value;
            const details = document.getElementById("details").value;
            document.getElementById("loading").style.display = "block";
            google.colab.kernel.invokeFunction("notebook.generate_project", [learningObjectives, subjects, details], {});
        }
    </script>
    """
    display(HTML(html))

# Callback para generar el proyecto
def generate_project(learning_objectives, subjects, details):
    if 'mistral_api_key' not in globals() or not mistral_api_key:
        print("⚠️ Error: No se encontró una API key válida. Por favor, configura tu API key.")
        return

    # Usar el agente para estructurar el prompt
    agente_contexto = agente.crear_contexto({
        "Objetivos de Aprendizaje": learning_objectives,
        "Asignaturas": subjects,
        "Detalles Adicionales": details
    })

    # Crear el prompt para Mistral usando el agente
    prompt = f"""
    {agente_contexto}
    Sigue el estándar del Gold Standard de PBLWorks y proporciona:
    - Título del Proyecto
    - Pregunta guía
    - Fase 1: Construir Conocimiento, Comprensión y Habilidades
    - Fase 2: Desarrollar y Evaluar Productos y Respuestas
    - Fase 3: Presentar Productos y Respuestas

    Nota: Asegúrate de incluir elementos de autenticidad, reflexión y retroalimentación.
    """

    # Llamar a la API de Mistral
    headers = {
        "Authorization": f"Bearer {mistral_api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "mistral-large-latest",
        "messages": [{"role": "user", "content": prompt}]
    }
    response = requests.post("https://api.mistral.ai/v1/chat/completions", json=data, headers=headers)

    # Procesar respuesta
    if response.status_code == 200:
        project = response.json().get("choices", [{}])[0].get("message", {}).get("content", "No se generó contenido.")
        # Mostrar el proyecto en formato Markdown
        display(Markdown(f"# Proyecto Generado\n\n{project}"))
        display(HTML("<script>document.getElementById('loading').style.display = 'none';</script>"))
    else:
        print(f"❌ Error al generar el proyecto: {response.status_code} - {response.text}")
        display(HTML("<script>document.getElementById('loading').style.display = 'none';</script>"))

# Registrar la función en Colab
output.register_callback("notebook.generate_project", generate_project)

# Solicitar detalles del proyecto al usuario
request_project_details()


# Proyecto Generado

### Agente de Diseño de Proyectos ABP para Docentes

**Título del Proyecto:** "Filosofía y Matemáticas en Acción: Resolviendo Dilemas Éticos y Optimizando Soluciones en un Mundo Global"

**Pregunta Guía:** "¿Cómo podemos utilizar la filosofía y las matemáticas para abordar dilemas éticos y optimizar soluciones en un mundo globalizado?"

---

### Fase 1: Construir Conocimiento, Comprensión y Habilidades

**Objetivo:** Introducir a los estudiantes a los conceptos filosóficos, matemáticos e inglés necesarios para abordar la pregunta guía.

**Actividades y Recursos:**

1. **Introducción a la Filosofía:**
   - Lectura y análisis de textos filosóficos clásicos y contemporáneos.
   - Discusión en grupo sobre dilemas éticos y su relevancia en la sociedad actual.
   - Talleres de debate y argumentación filosófica.

2. **Desarrollo de Habilidades Matemáticas:**
   - Clases teóricas y prácticas sobre funciones matemáticas avanzadas (polinomios, logaritmos).
   - Ejercicios de resolución de problemas y modelado matemático.
   - Introducción a métodos estadísticos y análisis de datos.

3. **Comprensión y Comunicación en Inglés:**
   - Lectura y comprensión de textos informativos y argumentativos en inglés.
   - Talleres de redacción de informes y ensayos en inglés.
   - Prácticas de comunicación oral y escrita en inglés.

**Evaluación:**
- Evaluación formativa a través de discusiones en clase, ejercicios prácticos y talleres.
- Retroalimentación continua para asegurar la comprensión y el desarrollo de habilidades.

---

### Fase 2: Desarrollar y Evaluar Productos y Respuestas

**Objetivo:** Aplicar el conocimiento adquirido para desarrollar soluciones innovadoras a problemas reales.

**Actividades y Recursos:**

1. **Proyecto Interdisciplinario:**
   - Formación de grupos de trabajo para abordar un dilema ético específico.
   - Uso de herramientas digitales para modelar y resolver problemas matemáticos relacionados con el dilema.
   - Recolección y análisis de datos relevantes utilizando métodos estadísticos.

2. **Debate y Argumentación:**
   - Preparación de debates sobre dilemas éticos, utilizando conceptos filosóficos y matemáticos.
   - Presentación de argumentos sostenidos con fundamentos teóricos.

3. **Redacción y Comunicación:**
   - Redacción de informes o ensayos en inglés sobre el proyecto interdisciplinario.
   - Preparación de presentaciones en inglés para comunicar los resultados del proyecto.

**Evaluación:**
- Evaluación sumativa de los productos desarrollados (informes, ensayos, presentaciones).
- Autoevaluación y retroalimentación entre pares para mejorar los productos finales.

---

### Fase 3: Presentar Productos y Respuestas

**Objetivo:** Compartir los resultados del proyecto con la comunidad educativa y reflexionar sobre el proceso de aprendizaje.

**Actividades y Recursos:**

1. **Presentación de Proyectos:**
   - Presentación de los proyectos interdisciplinarios en un evento escolar o comunitario.
   - Uso de herramientas digitales para apoyar la presentación (ppt, videos, gráficos).

2. **Reflexión y Retroalimentación:**
   - Sesiones de reflexión sobre el proceso de aprendizaje y los desafíos enfrentados.
   - Retroalimentación de los docentes y la comunidad educativa sobre los proyectos presentados.

3. **Publicación de Resultados:**
   - Publicación de los informes y ensayos en una plataforma digital o en un boletín escolar.
   - Difusión de los resultados del proyecto en redes sociales y otros medios de comunicación.

**Evaluación:**
- Evaluación final del proceso de aprendizaje y de los productos presentados.
- Reflexión sobre las habilidades desarrolladas y su aplicación en contextos reales.

---

### Elementos de Autenticidad, Reflexión y Retroalimentación

**Autenticidad:**
- Los proyectos abordan problemas reales y relevantes para los estudiantes.
- Uso de herramientas y métodos que se utilizan en el mundo profesional.

**Reflexión:**
- Sesiones regulares de reflexión sobre el proceso de aprendizaje.
- Diarios de reflexión y autoevaluación.

**Retroalimentación:**
- Retroalimentación continua de los docentes y entre pares.
- Evaluación formativa y sumativa para asegurar el desarrollo de habilidades y conocimientos.

---

Este proyecto sigue el Project Path y está alineado con los principios del Gold Standard de PBLWorks, garantizando su efectividad pedagógica y fomentando habilidades del siglo XXI y la innovación pedagógica.

# Evaluar proyecto por parte del usuario

In [19]:
from IPython.display import display, HTML, Markdown

# Solicitar evaluación del proyecto
def request_project_evaluation():
    html = """
    <div style="font-family: Arial, sans-serif; padding: 10px; max-width: 600px; margin: auto;">
        <h3 style="color: #2c3e50;">Evaluación del Proyecto Generado</h3>
        <p>Responde las siguientes preguntas clave para evaluar el proyecto generado:</p>

        <label>1. ¿El proyecto incluye una pregunta o desafío claro y desafiante?</label><br>
        <textarea id="q1" style="width: 100%; height: 60px; border: 1px solid #ccc; border-radius: 5px; padding: 5px;"></textarea><br><br>

        <label>2. ¿El proyecto permite una investigación sostenida y rigurosa?</label><br>
        <textarea id="q2" style="width: 100%; height: 60px; border: 1px solid #ccc; border-radius: 5px; padding: 5px;"></textarea><br><br>

        <label>3. ¿Se observa autenticidad en el proyecto?</label><br>
        <textarea id="q3" style="width: 100%; height: 60px; border: 1px solid #ccc; border-radius: 5px; padding: 5px;"></textarea><br><br>

        <label>4. ¿El proyecto otorga voz y elección a los estudiantes?</label><br>
        <textarea id="q4" style="width: 100%; height: 60px; border: 1px solid #ccc; border-radius: 5px; padding: 5px;"></textarea><br><br>

        <label>5. ¿Se promueve la reflexión durante el proyecto?</label><br>
        <textarea id="q5" style="width: 100%; height: 60px; border: 1px solid #ccc; border-radius: 5px; padding: 5px;"></textarea><br><br>

        <label>6. ¿Incluye oportunidades para crítica y revisión?</label><br>
        <textarea id="q6" style="width: 100%; height: 60px; border: 1px solid #ccc; border-radius: 5px; padding: 5px;"></textarea><br><br>

        <label>7. ¿Los productos generados se comparten públicamente?</label><br>
        <textarea id="q7" style="width: 100%; height: 60px; border: 1px solid #ccc; border-radius: 5px; padding: 5px;"></textarea><br><br>

        <button style="background-color: #3498db; color: white; padding: 10px 15px; border: none; border-radius: 5px; cursor: pointer;"
                onclick="submitEvaluation()">Enviar Evaluación</button>
        <div id="loading" style="display: none; margin-top: 20px; font-weight: bold; color: #3498db;">Procesando evaluación... Por favor, espera.</div>
    </div>
    <script>
        function submitEvaluation() {
            document.getElementById('loading').style.display = 'block';
            const responses = {
                q1: document.getElementById('q1').value,
                q2: document.getElementById('q2').value,
                q3: document.getElementById('q3').value,
                q4: document.getElementById('q4').value,
                q5: document.getElementById('q5').value,
                q6: document.getElementById('q6').value,
                q7: document.getElementById('q7').value,
            };
            google.colab.kernel.invokeFunction("notebook.evaluate_project", [responses], {});
        }
    </script>
    """
    display(HTML(html))

# Callback para evaluar el proyecto con Mistral
def evaluate_project_with_mistral(responses):
    print("Respuestas recibidas:", responses)  # Verifica el contenido de responses

    # Crear el prompt para la evaluación
    prompt = f"""
    Evaluar el siguiente proyecto de aprendizaje basado en proyectos (ABP) de acuerdo con el Gold Standard.
    Responde las siguientes preguntas clave basándote en las respuestas proporcionadas:

    1. ¿El proyecto incluye una pregunta o desafío claro y desafiante? Respuesta: {responses.get('q1', 'No proporcionado')}
    2. ¿El proyecto permite una investigación sostenida y rigurosa? Respuesta: {responses.get('q2', 'No proporcionado')}
    3. ¿Se observa autenticidad en el proyecto? Respuesta: {responses.get('q3', 'No proporcionado')}
    4. ¿El proyecto otorga voz y elección a los estudiantes? Respuesta: {responses.get('q4', 'No proporcionado')}
    5. ¿Se promueve la reflexión durante el proyecto? Respuesta: {responses.get('q5', 'No proporcionado')}
    6. ¿Incluye oportunidades para crítica y revisión? Respuesta: {responses.get('q6', 'No proporcionado')}
    7. ¿Los productos generados se comparten públicamente? Respuesta: {responses.get('q7', 'No proporcionado')}

    Realiza un análisis general sobre la calidad pedagógica del proyecto y ofrece sugerencias de mejora.
    """

    # Llamar a la API de Mistral
    headers = {
        "Authorization": f"Bearer {mistral_api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "mistral-large-latest",
        "messages": [{"role": "user", "content": prompt}]
    }
    response = requests.post("https://api.mistral.ai/v1/chat/completions", json=data, headers=headers)

    if response.status_code == 200:
        analysis = response.json().get("choices", [{}])[0].get("message", {}).get("content", "No se generó contenido.")
        display(Markdown(f"# Evaluación del Proyecto\n\n{analysis}"))
    else:
        display(Markdown(f"❌ Error: {response.status_code} - {response.text}"))

# Registrar el callback directamente
output.register_callback("notebook.evaluate_project", evaluate_project_with_mistral)

# Solicitar evaluación del proyecto
request_project_evaluation()


Respuestas recibidas: {'q1': 'Sí', 'q2': 'Si', 'q3': 'NO', 'q4': 'puede ser, aunque faltan espacios para los estudiantes', 'q5': 'Se promueve, pero deberíamos tener más preguntas', 'q6': 'Si', 'q7': 'Si'}


# Evaluación del Proyecto

### Análisis General sobre la Calidad Pedagógica del Proyecto

El proyecto evaluado presenta varios elementos positivos que lo alinean con el Gold Standard para el Aprendizaje Basado en Proyectos (ABP). Sin embargo, también hay áreas que requieren mejoras para alcanzar una calidad pedagógica óptima. A continuación, se detalla un análisis basado en las respuestas proporcionadas:

#### Elementos Positivos:

1. **Pregunta o Desafío Claro y Desafiante**:
   - El proyecto incluye una pregunta o desafío claro y desafiante, lo cual es esencial para motivar a los estudiantes y proporcionar una dirección clara para el aprendizaje.

2. **Investigación Sostenida y Rigurosa**:
   - El proyecto permite una investigación sostenida y rigurosa, lo que fomenta habilidades de investigación y pensamiento crítico en los estudiantes.

3. **Crítica y Revisión**:
   - El proyecto incluye oportunidades para la crítica y revisión, lo cual es crucial para el desarrollo de habilidades de autoevaluación y mejora continua.

4. **Productos Compartidos Públicamente**:
   - Los productos generados se comparten públicamente, lo que añade un nivel adicional de autenticidad y responsabilidad, además de proporcionar una audiencia real para el trabajo de los estudiantes.

#### Áreas de Mejora:

1. **Autenticidad**:
   - El proyecto carece de autenticidad, lo cual es un aspecto fundamental del ABP. La autenticidad implica que el proyecto tenga un propósito y una relevancia real para los estudiantes y el mundo exterior. Este es un área crítica que necesita ser abordada para que el proyecto sea más significativo y motivador para los estudiantes.

2. **Voz y Elección de los Estudiantes**:
   - Aunque el proyecto otorga voz y elección a los estudiantes, faltan espacios suficientes para que los estudiantes tomen decisiones y personalicen su aprendizaje. Ampliar estas oportunidades puede aumentar el compromiso y el sentido de propiedad sobre el proyecto.

3. **Reflexión**:
   - La reflexión se promueve, pero se necesitan más preguntas para guiar a los estudiantes a través del proceso reflexivo. La reflexión es esencial para el aprendizaje profundo y la metacognición, y debe ser más estructurada y frecuente.

### Sugerencias de Mejora:

1. **Aumentar la Autenticidad**:
   - **Conectar con el Mundo Real**: Identificar problemas o necesidades reales en la comunidad o el mundo laboral que el proyecto pueda abordar.
   - **Colaboraciones Externas**: Involucrar a expertos, organizaciones o miembros de la comunidad que puedan proporcionar contexto y relevancia adicional al proyecto.

2. **Ampliar la Voz y Elección de los Estudiantes**:
   - **Diseño Colaborativo**: Involucrar a los estudiantes desde el inicio en el diseño del proyecto, permitiéndoles influir en la pregunta o desafío, los métodos de investigación y los productos finales.
   - **Espacios de Decisión**: Crear momentos específicos en el proyecto donde los estudiantes puedan tomar decisiones clave, como elegir los métodos de investigación, los recursos a utilizar o los formatos de presentación.

3. **Mejorar la Reflexión**:
   - **Preguntas Guía**: Desarrollar un conjunto de preguntas guía que los estudiantes puedan utilizar en diferentes etapas del proyecto para reflexionar sobre su proceso de aprendizaje y sus resultados.
   - **Diarios de Reflexión**: Implementar diarios de reflexión donde los estudiantes puedan registrar sus pensamientos, desafíos y logros a lo largo del proyecto.
   - **Sesiones de Reflexión en Grupo**: Organizar sesiones de reflexión en grupo donde los estudiantes puedan compartir y discutir sus experiencias y aprendizajes.

### Conclusión

El proyecto evaluado tiene una base sólida con varios elementos alineados con el Gold Standard del ABP. Sin embargo, para alcanzar una calidad pedagógica óptima, es necesario abordar las áreas de autenticidad, voz y elección de los estudiantes, y reflexión. Implementando las sugerencias de mejora, el proyecto puede convertirse en una experiencia de aprendizaje más rica y significativa para los estudiantes.

# Propuestas de mejora

In [20]:
from IPython.display import display, HTML, Markdown
import requests

# Variable global para almacenar la opción seleccionada
selected_option = None

# Función para generar opciones de proyectos mejorados
def generate_improved_projects(responses):
    global selected_option

    # Crear el prompt para Mistral
    prompt = f"""
    Basándote en la evaluación del proyecto proporcionada, crea tres opciones mejoradas para el proyecto original.
    Cada opción debe cumplir con los principios del Gold Standard y ofrecer enfoques distintos.
    Responde en el siguiente formato:

    ## OPCION 1
    [Descripción detallada del proyecto]

    ## OPCION 2
    [Descripción detallada del proyecto]

    ## OPCION 3
    [Descripción detallada del proyecto]
    """
    # Llamar a la API de Mistral
    headers = {
        "Authorization": f"Bearer {mistral_api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "mistral-large-latest",
        "messages": [{"role": "user", "content": prompt}]
    }
    response = requests.post("https://api.mistral.ai/v1/chat/completions", json=data, headers=headers)

    if response.status_code == 200:
        project_options = response.json().get("choices", [{}])[0].get("message", {}).get("content", "No se generó contenido.")
        display(Markdown(f"# Opciones de Proyecto Mejorado\n\n{project_options}"))
        display(HTML("""
        <div style="margin-top: 20px;">
            <p><b>Selecciona una opción:</b></p>
            <button onclick="selectOption(1)" style="background-color: #3498db; color: white; padding: 10px 15px; margin-right: 10px;">Opción 1</button>
            <button onclick="selectOption(2)" style="background-color: #3498db; color: white; padding: 10px 15px; margin-right: 10px;">Opción 2</button>
            <button onclick="selectOption(3)" style="background-color: #3498db; color: white; padding: 10px 15px;">Opción 3</button>
        </div>
        <script>
            function selectOption(option) {
                google.colab.kernel.invokeFunction("notebook.select_project_option", [option], {});
            }
        </script>
        """))
    else:
        display(Markdown(f"❌ Error al generar opciones: {response.status_code}"))
        display(Markdown(f"Detalles del error: {response.text}"))

# Callback para guardar la opción seleccionada
def save_selected_option(option):
    global selected_option
    selected_option = option
    display(Markdown(f"✅ Has seleccionado la Opción {option}"))

# Registrar la función en Google Colab
output.register_callback("notebook.select_project_option", save_selected_option)

# Ejemplo: Llamar a la función con datos simulados
mock_responses = {
    "q1": "Sí, el proyecto tiene una pregunta desafiante.",
    "q2": "El proyecto permite investigaciones rigurosas.",
    "q3": "El proyecto es auténtico.",
    "q4": "Los estudiantes tienen voz y elección.",
    "q5": "Promueve reflexión constante.",
    "q6": "Incluye oportunidades para revisión.",
    "q7": "Los productos se comparten públicamente."
}

# Generar opciones de proyecto mejorado
generate_improved_projects(mock_responses)


# Opciones de Proyecto Mejorado

## OPCIÓN 1
### Descripción detallada del proyecto

**Título del Proyecto:** Implementación de Paneles Solares en Comunidades Rurales

**Descripción:**
Este proyecto se centra en la instalación de paneles solares en comunidades rurales que actualmente no tienen acceso a la electricidad. La energía solar no solo proporcionará una fuente de energía limpia y renovable, sino que también mejorará la calidad de vida de los residentes al permitir el acceso a tecnologías modernas como iluminación, refrigeración y comunicaciones.

**Cumplimiento con los Principios del Gold Standard:**
1. **Desarrollo Sostenible:** Promueve el acceso a la energía limpia y mejora la calidad de vida en comunidades rurales.
2. **Reducción de Emisiones:** Reduce la dependencia de combustibles fósiles y disminuye las emisiones de CO2.
3. **Participación Comunitaria:** Involucra a la comunidad local en la planificación y ejecución del proyecto, asegurando que se adapte a sus necesidades y expectativas.
4. **Transparencia y Verificación:** Incluye un sistema de monitoreo y verificación para asegurar que los beneficios ambientales y sociales sean reales y medibles.

## OPCIÓN 2
### Descripción detallada del proyecto

**Título del Proyecto:** Rehabilitación de Bosques Degradados

**Descripción:**
Este proyecto se enfoca en la rehabilitación de bosques degradados mediante la plantación de árboles nativos y la implementación de prácticas de manejo sostenible. La reforestación no solo capturará carbono, sino que también restaurará hábitats naturales, mejorará la biodiversidad y proporcionará beneficios económicos a las comunidades locales a través de la creación de empleos en actividades de conservación y ecoturismo.

**Cumplimiento con los Principios del Gold Standard:**
1. **Desarrollo Sostenible:** Contribuye a la restauración de ecosistemas y fomenta el desarrollo económico local.
2. **Reducción de Emisiones:** La reforestación actúa como un sumidero de carbono, mitigando el cambio climático.
3. **Participación Comunitaria:** Involucra a las comunidades locales en la planificación y ejecución del proyecto, asegurando que se beneficien directamente de la reforestación.
4. **Transparencia y Verificación:** Establece un sistema de monitoreo para medir el crecimiento de los árboles y el secuestro de carbono, garantizando la transparencia y la verificación de los resultados.

## OPCIÓN 3
### Descripción detallada del proyecto

**Título del Proyecto:** Sistema de Gestión de Residuos Sostenible

**Descripción:**
Este proyecto propone la implementación de un sistema de gestión de residuos sostenible en una ciudad urbana. Incluye la separación en origen, reciclaje, compostaje y la conversión de residuos orgánicos en biogás. Este enfoque integral no solo reduce la cantidad de residuos que terminan en vertederos, sino que también genera energía renovable y productos reciclados, promoviendo una economía circular.

**Cumplimiento con los Principios del Gold Standard:**
1. **Desarrollo Sostenible:** Mejora la gestión de residuos, reduce la contaminación y promueve la economía circular.
2. **Reducción de Emisiones:** Disminuye las emisiones de metano de los vertederos y reduce la necesidad de energía derivada de combustibles fósiles.
3. **Participación Comunitaria:** Involucra a la comunidad en programas de educación y sensibilización sobre la gestión de residuos, asegurando su participación activa.
4. **Transparencia y Verificación:** Implementa un sistema de monitoreo para medir la reducción de residuos y las emisiones evitadas, garantizando la transparencia y la verificación de los resultados.

In [22]:
from IPython.display import display, HTML, Markdown
import requests

# Variable global para almacenar la opción seleccionada
selected_option = None

# Solicitar evaluación del proyecto
def request_project_evaluation():
    html = """
    <div style="font-family: Arial, sans-serif; padding: 10px; max-width: 600px; margin: auto;">
        <h3 style="color: #2c3e50;">Evaluación del Proyecto Generado</h3>
        <p>Responde las siguientes preguntas clave para evaluar el proyecto generado:</p>

        <label>1. ¿El proyecto incluye una pregunta o desafío claro y desafiante?</label><br>
        <textarea id="q1" style="width: 100%; height: 60px; border: 1px solid #ccc; border-radius: 5px; padding: 5px;"></textarea><br><br>

        <label>2. ¿El proyecto permite una investigación sostenida y rigurosa?</label><br>
        <textarea id="q2" style="width: 100%; height: 60px; border: 1px solid #ccc; border-radius: 5px; padding: 5px;"></textarea><br><br>

        <label>3. ¿Se observa autenticidad en el proyecto?</label><br>
        <textarea id="q3" style="width: 100%; height: 60px; border: 1px solid #ccc; border-radius: 5px; padding: 5px;"></textarea><br><br>

        <label>4. ¿El proyecto otorga voz y elección a los estudiantes?</label><br>
        <textarea id="q4" style="width: 100%; height: 60px; border: 1px solid #ccc; border-radius: 5px; padding: 5px;"></textarea><br><br>

        <label>5. ¿Se promueve la reflexión durante el proyecto?</label><br>
        <textarea id="q5" style="width: 100%; height: 60px; border: 1px solid #ccc; border-radius: 5px; padding: 5px;"></textarea><br><br>

        <label>6. ¿Incluye oportunidades para crítica y revisión?</label><br>
        <textarea id="q6" style="width: 100%; height: 60px; border: 1px solid #ccc; border-radius: 5px; padding: 5px;"></textarea><br><br>

        <label>7. ¿Los productos generados se comparten públicamente?</label><br>
        <textarea id="q7" style="width: 100%; height: 60px; border: 1px solid #ccc; border-radius: 5px; padding: 5px;"></textarea><br><br>

        <button style="background-color: #3498db; color: white; padding: 10px 15px; border: none; border-radius: 5px; cursor: pointer;"
                onclick="submitEvaluation()">Enviar Evaluación</button>
        <div id="loading" style="display: none; margin-top: 20px; font-weight: bold; color: #3498db;">Procesando evaluación... Por favor, espera.</div>
    </div>
    <script>
        function submitEvaluation() {
            document.getElementById('loading').style.display = 'block';
            const responses = {
                q1: document.getElementById('q1').value,
                q2: document.getElementById('q2').value,
                q3: document.getElementById('q3').value,
                q4: document.getElementById('q4').value,
                q5: document.getElementById('q5').value,
                q6: document.getElementById('q6').value,
                q7: document.getElementById('q7').value,
            };
            google.colab.kernel.invokeFunction("notebook.evaluate_project", [responses], {});
        }
    </script>
    """
    display(HTML(html))

# Función para generar opciones de proyectos mejorados
def generate_improved_projects(responses):
    global selected_option

    # Crear el prompt para Mistral
    prompt = f"""
    Basándote en la evaluación del proyecto proporcionada, crea tres opciones mejoradas para el proyecto original.
    Cada opción debe cumplir con los principios del Gold Standard y ofrecer enfoques distintos.
    Responde en el siguiente formato:

    ## OPCION 1
    [Descripción detallada del proyecto]

    ## OPCION 2
    [Descripción detallada del proyecto]

    ## OPCION 3
    [Descripción detallada del proyecto]
    """
    # Llamar a la API de Mistral
    headers = {
        "Authorization": f"Bearer {mistral_api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "mistral-large-latest",
        "messages": [{"role": "user", "content": prompt}]
    }
    response = requests.post("https://api.mistral.ai/v1/chat/completions", json=data, headers=headers)

    if response.status_code == 200:
        project_options = response.json().get("choices", [{}])[0].get("message", {}).get("content", "No se generó contenido.")
        display(Markdown(f"# Opciones de Proyecto Mejorado\n\n{project_options}"))
        display(HTML("""
        <div style="margin-top: 20px;">
            <p><b>Selecciona una opción:</b></p>
            <button onclick="selectOption(1)" style="background-color: #3498db; color: white; padding: 10px 15px; margin-right: 10px;">Opción 1</button>
            <button onclick="selectOption(2)" style="background-color: #3498db; color: white; padding: 10px 15px; margin-right: 10px;">Opción 2</button>
            <button onclick="selectOption(3)" style="background-color: #3498db; color: white; padding: 10px 15px;">Opción 3</button>
        </div>
        <script>
            function selectOption(option) {
                google.colab.kernel.invokeFunction("notebook.select_project_option", [option], {});
            }
        </script>
        """))
    else:
        display(Markdown(f"❌ Error al generar opciones: {response.status_code}"))
        display(Markdown(f"Detalles del error: {response.text}"))

# Callback para manejar las respuestas y generar proyectos mejorados
def handle_evaluation_and_generate_projects(responses):
    print("Respuestas recibidas:", responses)
    generate_improved_projects(responses)

# Callback para guardar la opción seleccionada
def save_selected_option(option):
    global selected_option
    selected_option = option
    display(Markdown(f"✅ Has seleccionado la Opción {option}"))

# Registrar funciones en Google Colab
output.register_callback("notebook.evaluate_project", handle_evaluation_and_generate_projects)
output.register_callback("notebook.select_project_option", save_selected_option)

# Solicitar evaluación del proyecto
request_project_evaluation()


Respuestas recibidas: {'q1': 'Si', 'q2': 'No', 'q3': 'SI', 'q4': 'Si', 'q5': 'Falya sumar una actividad para eso', 'q6': 'Necesito preguntas', 'q7': 'Si'}


# Opciones de Proyecto Mejorado

## OPCIÓN 1
**Proyecto de Energía Solar Comunitaria**

**Descripción Detallada:**
Este proyecto se centra en la instalación de paneles solares en comunidades rurales para proporcionar electricidad limpia y accesible. La iniciativa incluye la formación de una cooperativa comunitaria que gestione y mantenga los paneles solares, asegurando que los beneficios económicos se reinviertan en la comunidad. Además, se implementará un programa de educación y capacitación para que los residentes locales aprendan sobre la tecnología solar y puedan participar en el mantenimiento y la expansión del proyecto.

**Cumplimiento con el Gold Standard:**
- **Reducción de Emisiones:** Al reemplazar fuentes de energía tradicionales como el carbón y el diesel con energía solar, se reducirán significativamente las emisiones de CO2.
- **Beneficios Sociales:** La cooperativa comunitaria asegurará que los ingresos generados se utilicen para mejorar la infraestructura local y financiar proyectos comunitarios.
- **Transparencia y Participación:** Se establecerá un comité de supervisión compuesto por miembros de la comunidad para garantizar la transparencia y la participación continua en la toma de decisiones.

## OPCIÓN 2
**Proyecto de Agricultura Sostenible y Reforestación**

**Descripción Detallada:**
Este proyecto combina técnicas de agricultura sostenible con un programa de reforestación para restaurar ecosistemas degradados y mejorar la productividad agrícola. Se implementarán prácticas como la agroforestería, la rotación de cultivos y el uso de fertilizantes orgánicos. Además, se llevará a cabo un programa de plantación de árboles nativos en áreas deforestadas para restaurar la biodiversidad y mejorar la captura de carbono.

**Cumplimiento con el Gold Standard:**
- **Reducción de Emisiones:** La reforestación y las prácticas agrícolas sostenibles aumentarán la capacidad de captura de carbono del suelo y la vegetación.
- **Beneficios Sociales:** Los agricultores locales recibirán capacitación y apoyo para adoptar prácticas sostenibles, mejorando su productividad y sus ingresos.
- **Transparencia y Participación:** Se establecerá un sistema de monitoreo y evaluación para rastrear el progreso del proyecto y asegurar la participación de la comunidad en todas las etapas.

## OPCIÓN 3
**Proyecto de Gestión Integral de Residuos**

**Descripción Detallada:**
Este proyecto se enfoca en la implementación de un sistema de gestión integral de residuos en una ciudad, incluyendo la recolección, clasificación y reciclaje de materiales. Se establecerá una planta de compostaje para procesar residuos orgánicos y reducir la cantidad de desechos enviados a vertederos. Además, se lanzarán campañas de educación pública para promover la reducción de residuos y el reciclaje entre los ciudadanos.

**Cumplimiento con el Gold Standard:**
- **Reducción de Emisiones:** La reducción de residuos en vertederos disminuirá las emisiones de metano, un potente gas de efecto invernadero.
- **Beneficios Sociales:** La creación de empleos en la gestión de residuos y el reciclaje mejorará la economía local y promoverá la sostenibilidad.
- **Transparencia y Participación:** Se formará un comité ciudadano para supervisar el proyecto y asegurar la transparencia en la gestión de residuos, fomentando la participación comunitaria.

# Versión final

In [ ]:
!apt-get install wkhtmltopdf
!pip install pdfkit

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  avahi-daemon bind9-host bind9-libs geoclue-2.0 glib-networking glib-networking-common
  glib-networking-services gsettings-desktop-schemas iio-sensor-proxy libavahi-core7 libavahi-glib1
  libdaemon0 libevdev2 libfontenc1 libgudev-1.0-0 libhyphen0 libinput-bin libinput10
  libjson-glib-1.0-0 libjson-glib-1.0-common liblmdb0 libmaxminddb0 libmbim-glib4 libmbim-proxy
  libmd4c0 libmm-glib0 libmtdev1 libnl-genl-3-200 libnotify4 libnss-mdns libproxy1v5 libqmi-glib5
  libqmi-proxy libqt5core5a libqt5dbus5 libqt5gui5 libqt5network5 libqt5positioning5
  libqt5printsupport5 libqt5qml5 libqt5qmlmodels5 libqt5quick5 libqt5sensors5 libqt5svg5
  libqt5webchannel5 libqt5webkit5 libqt5widgets5 libsoup2.4-1 libsoup2.4-common libudev1
  libwacom-bin libwacom-common libwacom9 libwoff1 libxcb-icccm4 libxcb-image0 libxcb-keysyms1
  libxcb-render-util0 libx

In [ ]:
import requests
from IPython.display import display, Markdown
import pdfkit

# Función para generar retroalimentación usando Mistral
def generate_feedback(responses):
    if 'mistral_api_key' not in globals() or not mistral_api_key:
        print("Error: No se encontró una API key válida. Por favor, configura tu API key.")
        return None

    # Crear el prompt basado en las respuestas
    prompt = "Evalúa un proyecto basado en las siguientes respuestas:\n"
    for question, answer in responses.items():
        prompt += f"{question}: {answer}\n"

    prompt += "\nProporciona sugerencias para mejorar el proyecto según el estándar Gold Standard de PBL."

    # Llamar a la API de Mistral
    headers = {
        "Authorization": f"Bearer {mistral_api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "mistral-large-latest",
        "messages": [{"role": "user", "content": prompt}]
    }

    try:
        response = requests.post("https://api.mistral.ai/v1/chat/completions", json=data, headers=headers)
        response.raise_for_status()
        feedback = response.json()
        if "choices" in feedback and len(feedback["choices"]) > 0:
            return feedback["choices"][0]["message"]["content"]
        else:
            print("La API no devolvió contenido.")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error durante la solicitud: {e}")
        return None

# Función para mostrar la retroalimentación en Markdown
def display_feedback_in_markdown(feedback):
    if feedback:
        display(Markdown(f"# Retroalimentación Mejorada\n\n{feedback}"))
    else:
        print("No hay retroalimentación para mostrar.")


In [ ]:
# Función para exportar retroalimentación a PDF
def export_feedback_to_pdf(feedback, filename="evaluacion_proyecto.pdf"):
    # Convertir la retroalimentación en formato HTML
    html_content = f"""
    <html>
    <head>
        <style>
            body {{
                font-family: Arial, sans-serif;
                line-height: 1.6;
            }}
            h1 {{
                color: #007BFF;
            }}
            pre {{
                background: #f8f9fa;
                padding: 10px;
                border-radius: 5px;
            }}
        </style>
    </head>
    <body>
        <h1>Retroalimentación Mejorada</h1>
        <pre>{feedback}</pre>
    </body>
    </html>
    """
    # Guardar el contenido como PDF
    try:
        pdfkit.from_string(html_content, filename)
        print(f"PDF generado: {filename}")
    except Exception as e:
        print(f"Error al generar el PDF: {e}")


In [ ]:
# Simular respuestas del formulario
responses = {
    "q1": "Sí, la pregunta guía es desafiante.",
    "q2": "Sí, el proyecto incluye actividades de investigación rigurosa.",
    "q3": "El proyecto parece auténtico y relevante.",
    "q4": "Los estudiantes tienen voz en la elección de las actividades.",
    "q5": "Sí, se incluyen reflexiones a lo largo del proyecto.",
    "q6": "Hay oportunidades para revisión y crítica constructiva.",
    "q7": "Los productos se presentan públicamente."
}

# Generar retroalimentación
feedback = generate_feedback(responses)

# Validar y procesar retroalimentación
if feedback:
    # Mostrar en Markdown
    display_feedback_in_markdown(feedback)

    # Exportar a PDF
    export_feedback_to_pdf(feedback)
else:
    print("No se generó retroalimentación. Revisa los errores anteriores.")


# Retroalimentación Mejorada

El estándar Gold Standard de Project Based Learning (PBL) se centra en seis criterios clave: desafío y autenticidad, voz y elección, reflexión y crítica, productos públicos, y procesos de revisión y mejora. Basado en las respuestas proporcionadas, aquí tienes una evaluación y algunas sugerencias para mejorar el proyecto según estos criterios:

1. **Desafío y Autenticidad**:
   - **Evaluación**: La pregunta guía es desafiante, y el proyecto parece auténtico y relevante.
   - **Sugerencia**: Asegúrate de que la pregunta guía no solo sea desafiante, sino también relevante y significativa para los estudiantes. Puedes enriquecerla aún más alineándola con problemas del mundo real o temas de actualidad.

2. **Investigación Rigurosa**:
   - **Evaluación**: El proyecto incluye actividades de investigación rigurosa.
   - **Sugerencia**: Asegúrate de que la investigación no solo sea rigurosa, sino también diversa. Incorpora múltiples fuentes y perspectivas para enriquecer el proceso de aprendizaje.

3. **Voz y Elección**:
   - **Evaluación**: Los estudiantes tienen voz en la elección de las actividades.
   - **Sugerencia**: Amplía las opciones de elección para los estudiantes, permitiéndoles no solo elegir actividades, sino también influir en la dirección del proyecto y en la forma en que se presentan los productos finales.

4. **Reflexión y Crítica**:
   - **Evaluación**: Se incluyen reflexiones a lo largo del proyecto.
   - **Sugerencia**: Fomenta la reflexión continua y profunda. Utiliza herramientas de reflexión como diarios de aprendizaje, discusiones en grupo y autoevaluaciones para que los estudiantes puedan analizar su propio proceso de aprendizaje y el de sus compañeros.

5. **Revisión y Mejora**:
   - **Evaluación**: Hay oportunidades para revisión y crítica constructiva.
   - **Sugerencia**: Establece un proceso estructurado de revisión por pares y autoevaluación. Utiliza rúbricas claras y específicas para guiar la revisión y asegúrate de que los estudiantes reciban retroalimentación constructiva y oportunidades para mejorar su trabajo.

6. **Productos Públicos**:
   - **Evaluación**: Los productos se presentan públicamente.
   - **Sugerencia**: Asegúrate de que la presentación pública sea significativa y tenga un impacto. Considera diferentes formatos de presentación, como exposiciones, presentaciones en línea, publicaciones en blogs o revistas escolares, para maximizar el impacto y la audiencia.

### Sugerencias Adicionales:
- **Integración de Tecnologías**: Utiliza herramientas tecnológicas para facilitar el proceso de investigación, colaboración y presentación de los productos. Esto puede incluir plataformas de gestión de proyectos, aplicaciones de colaboración en línea, y herramientas de presentación digital.
- **Colaboración y Trabajo en Equipo**: Fomenta la colaboración y el trabajo en equipo. Asegúrate de que los estudiantes trabajen juntos de manera efectiva, asignando roles claros y promoviendo la comunicación abierta.
- **Evaluación Formativa y Sumativa**: Implementa tanto evaluaciones formativas (durante el proyecto) como sumativas (al final del proyecto) para medir el progreso y el aprendizaje de los estudiantes.
- **Conexión con el Currículo**: Asegúrate de que el proyecto esté bien alineado con el currículo escolar y los objetivos de aprendizaje.

Implementando estas sugerencias, puedes mejorar aún más la calidad del proyecto y alinearlo más estrechamente con el estándar Gold Standard de PBL.

PDF generado: evaluacion_proyecto.pdf
